In [3]:
import os
import json

# Gebe hier den Pfad zu den Ordnern an, die die JSON-Dateien enthalten
folder_path = '/Users/arthurp/Documents/Dev/chats/trans/exported_chats_sorted/output_data'

# Eine Liste zum Speichern aller Chat-Daten
all_chats = []

# Funktion, um alle JSON-Dateien rekursiv zu finden und zu laden
def load_json_files(folder_path):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.json'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    chat_data = json.load(file)
                    all_chats.append(chat_data)

# Lade alle JSON-Dateien aus dem Hauptordner und seinen Unterordnern
load_json_files(folder_path)

# Speichere alle gesammelten Chats in einer neuen Datei
output_file_path = './combined_chats.json'
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(all_chats, output_file, ensure_ascii=False, indent=4)

print(f"Alle Chats wurden in {output_file_path} zusammengeführt.")

Alle Chats wurden in ./combined_chats.json zusammengeführt.


In [1]:
import json
import pandas as pd

def json_to_hdf5(json_file, hdf5_file):
    # Lade die JSON-Datei
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Leere Liste für alle Nachrichten
    all_data = []

    # Durchlaufe alle Chats in der JSON-Datei
    for chat in data:
        chat_id = chat["chat_id"]  # Chat-ID extrahieren
        for message in chat["messages"]:
            message_data = {
                "chat_id": chat_id,
                "sender_alias": message["sender_alias"],
                "message": message.get("message", ""),
                "message_deepl": message.get("message_deepl", ""),
                "timestamp": pd.to_datetime(message["timestamp"])  # Konvertiere Zeitstempel
            }
            all_data.append(message_data)

    # Erstelle DataFrame aus den Nachrichten
    df = pd.DataFrame(all_data)

    # Speichere den DataFrame als HDF5-Datei
    df.to_hdf(hdf5_file, key='chats', mode='w')  # 'chats' ist der Schlüssel

# Beispielaufruf: JSON in HDF5 konvertieren
json_file = 'json/combined_chats.json'  # Dein JSON-Dateipfad
hdf5_file = 'chats.h5'  # Der Ziel-HDF5-Dateipfad

json_to_hdf5(json_file, hdf5_file)


In [4]:
import json
import h5py
import pandas as pd
import numpy as np
from datetime import datetime
import argparse
import os

def convert_json_to_h5(json_file_path, h5_file_path):
    """
    Konvertiert eine JSON-Datei mit Chat-Daten in eine H5-Datei.
    
    Args:
        json_file_path (str): Pfad zur JSON-Datei
        h5_file_path (str): Pfad, wo die H5-Datei gespeichert werden soll
    """
    print(f"Lese JSON-Datei: {json_file_path}")
    
    # JSON-Datei einlesen
    with open(json_file_path, 'r', encoding='utf-8') as file:
        chat_data = json.load(file)
    
    print(f"Gefundene Chats: {len(chat_data)}")
    
    # H5-Datei erstellen
    with h5py.File(h5_file_path, 'w') as hf:
        # Durchlaufe jeden Chat
        for chat_idx, chat in enumerate(chat_data):
            chat_id = chat['chat_id']
            print(f"Verarbeite Chat {chat_idx+1}/{len(chat_data)}: {chat_id}")
            
            # Gruppe für diesen Chat erstellen
            chat_group = hf.create_group(chat_id)
            
            # Chat-Metadaten als Attribute hinzufügen
            chat_group.attrs['unique_sender_count'] = chat['unique_sender_count']
            chat_group.attrs['message_count'] = chat['message_count']
            chat_group.attrs['chat_name'] = chat_id.split(':')[-1] if ':' in chat_id else chat_id
            
            # Extrahiere Nachrichtendaten
            messages = chat['messages']
            if messages:
                # Datenstrukturen für H5-Datasets vorbereiten
                timestamps = []
                timestamp_strings = []
                sender_aliases = []
                message_texts = []
                message_ids = []
                message_deepl_texts = []
                has_deepl = False
                
                # Gehe durch jede Nachricht im Chat
                for msg in messages:
                    # Zeitstempel als Unix-Timestamp und als String speichern
                    try:
                        dt = datetime.strptime(msg['timestamp'], "%Y-%m-%d %H:%M:%S")
                        timestamps.append(dt.timestamp())
                        timestamp_strings.append(msg['timestamp'])
                    except:
                        timestamps.append(np.nan)
                        timestamp_strings.append(msg.get('timestamp', ''))
                    
                    # Sender und Nachrichtentext
                    sender_aliases.append(msg.get('sender_alias', ''))
                    message_texts.append(msg.get('message', ''))
                    message_ids.append(msg.get('message_id', -1))
                    
                    # Übersetzte Nachricht (wenn vorhanden)
                    if 'message_deepl' in msg:
                        has_deepl = True
                        message_deepl_texts.append(msg['message_deepl'])
                    else:
                        message_deepl_texts.append('')
                
                # Erstelle Datasets für die Nachrichtendaten
                dt_string = h5py.special_dtype(vlen=str)
                
                timestamps_dataset = chat_group.create_dataset('timestamp', data=timestamps)
                timestamp_str_dataset = chat_group.create_dataset('timestamp_str', data=timestamp_strings, dtype=dt_string)
                
                sender_dataset = chat_group.create_dataset('sender_alias', data=sender_aliases, dtype=dt_string)
                message_dataset = chat_group.create_dataset('message', data=message_texts, dtype=dt_string)
                id_dataset = chat_group.create_dataset('message_id', data=message_ids)
                
                if has_deepl:
                    deepl_dataset = chat_group.create_dataset('message_deepl', data=message_deepl_texts, dtype=dt_string)
    
    print(f"Konvertierung abgeschlossen. H5-Datei gespeichert unter: {h5_file_path}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Konvertiert eine JSON-Datei mit Chat-Daten in eine H5-Datei.")
    parser.add_argument("json_file", help="Pfad zur JSON-Datei")
    parser.add_argument("--output", "-o", help="Pfad zur Ausgabe-H5-Datei (Optional)", default=None)
    
    args = parser.parse_args()
    
    json_file_path = args.json_file
    
    if args.output:
        h5_file_path = args.output
    else:
        # Wenn kein Ausgabepfad angegeben ist, nutze den gleichen Namen wie die JSON-Datei
        h5_file_path = os.path.splitext(json_file_path)[0] + '.h5'
    
    convert_json_to_h5(json_file_path, h5_file_path)

usage: ipykernel_launcher.py [-h] [--output OUTPUT] json_file
ipykernel_launcher.py: error: the following arguments are required: json_file


SystemExit: 2

/opt/anaconda3/envs/myenv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
